In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of models.resnet_enas failed: Traceback (most recent call last):
  File "/home/tobias/virtualenv_environments/BmxNet/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/tobias/virtualenv_environments/BmxNet/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/tobias/virtualenv_environments/BmxNet/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tobias/virtualenv_environments/BmxNet/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/tobias/virtualenv_environments/BmxNet/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tobias/virtualenv_environments/BmxNet/lib/

In [15]:
import sys
sys.path.insert(0, "../")

import autogluon as ag
import mxnet as mx
import mxnet.gluon.nn as nn
import mxnet.ndarray as nd
import mxnet.gluon as gluon
from mxnet.gluon.nn import BinaryLayerConfig
from mxnet.gluon import ParameterDict
from mxnet import gpu

In [16]:
from models.meliusnet_enas import ImprovementBlockEnas
from models.resnet_enas import BasicBlockV1Enas
from utils.binary_layers import *

In [19]:
a = nd.array((3,3))

In [23]:
a.dtype

numpy.float32

In [21]:
a.det_sign().dtype

numpy.float32

In [4]:
block = BasicBlockV1Enas(2,1)
para = block.body[0].qconv.params
block.initialize()
x = nd.random.uniform(shape=(1,1,8,8))
y = block(x)

In [5]:
block2 = BasicBlockV1Enas(2,1, parameter_sharing_partner=block)
para = block.body[0].qconv.params
block2.initialize()
x = nd.random.uniform(shape=(1,1,8,8))
y = block2(x)

In [6]:
block.collect_params()

basicblockv1enas0_ (
  Parameter basicblockv1enas0_qconv0_weight (shape=(2, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_gamma (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_beta (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_running_mean (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_running_var (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_qconv1_weight (shape=(2, 2, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_gamma (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_beta (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_running_mean (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_running_var (shape=(2,), dtype=<class 'numpy.float32'>)
)

In [7]:
block2.params.prefix

'basicblockv1enas0_'

In [8]:
block2.collect_params()

basicblockv1enas0_ (
  Parameter basicblockv1enas0_qconv0_weight (shape=(2, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_gamma (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_beta (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_running_mean (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm0_running_var (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_qconv1_weight (shape=(2, 2, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_gamma (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_beta (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_running_mean (shape=(2,), dtype=<class 'numpy.float32'>)
  Parameter basicblockv1enas0_batchnorm1_running_var (shape=(2,), dtype=<class 'numpy.float32'>)
)

In [10]:
block2.collect_params().get('qconv0_weight').data() is block.collect_params().get('qconv0_weight').data()

True

In [10]:
block2.body[0].qconv.params

basicblockv1enas0_qconv0_ (
  Parameter basicblockv1enas0_qconv0_weight (shape=(2, 1, 3, 3), dtype=<class 'numpy.float32'>)
)

In [13]:
block2.body[0].qconv.params

basicblockv1enas0_qconv0_ (
  Parameter basicblockv1enas0_qconv0_weight (shape=(2, 1, 3, 3), dtype=<class 'numpy.float32'>)
)

In [25]:
block2.body[0].qconv.params.get('weight').data()


[[[[ 0.00982754 -0.06462044 -0.00859579]
   [ 0.01879837  0.06837233  0.0642529 ]
   [-0.05571373  0.02139064 -0.04075725]]]


 [[[ 0.01890824 -0.04741667  0.06934194]
   [ 0.02143516  0.01145905 -0.03453917]
   [-0.0119884  -0.00471649 -0.00354235]]]]
<NDArray 2x1x3x3 @cpu(0)>

In [18]:
block2.collect_params()['basicblockv1enas0_qconv0_weight'].data()


[[[[ 0.00982754 -0.06462044 -0.00859579]
   [ 0.01879837  0.06837233  0.0642529 ]
   [-0.05571373  0.02139064 -0.04075725]]]


 [[[ 0.01890824 -0.04741667  0.06934194]
   [ 0.02143516  0.01145905 -0.03453917]
   [-0.0119884  -0.00471649 -0.00354235]]]]
<NDArray 2x1x3x3 @cpu(0)>

In [21]:
block2.body[0].deconv.params.get['weight'].data() is block.collect_params().get('qconv0_weight').data()

AttributeError: 'BinaryConvolution' object has no attribute 'deconv'

In [138]:
block2.conv1_weights.get('weight').data()


[[[[ 0.03603098  0.01036446 -0.02595974]
   [ 0.06229289  0.02202464  0.00836839]
   [ 0.00242565  0.00905331 -0.00210481]]]


 [[[-0.06283614  0.0561627  -0.05542897]
   [ 0.00765031  0.01602133  0.04576063]
   [ 0.03242872  0.0315803  -0.03583546]]]]
<NDArray 2x1x3x3 @cpu(0)>

In [110]:
bconv.params

basicblockv1enas10_qconv0_ (
  Parameter basicblockv1enas10_qconv0_weight (shape=(2, 1, 3, 3), dtype=<class 'numpy.float32'>)
)

In [114]:
bconv.params.get('weight').data() is para.get('weight').data()

True

In [81]:
bconv.initialize()
y = tmp(x)

In [87]:
bconv.params.get('weight').data().dtype

numpy.float32

In [66]:
bconv.params.get('weight').data()[0] is block.collect_params().get('qconv0_weight').data()[0]

False

In [121]:
type(bconv.params.items())

odict_items

In [115]:
new_dict = ParameterDict()

In [120]:
new_dict.items()[0][1] = bconv.params.get('weight')

TypeError: 'odict_items' object does not support indexing

In [7]:
ibe = ImprovementBlockEnas(64,64)

AttributeError: 'BinaryConvolution' object has no attribute 'weight'

In [41]:
ibe.collect_params()

improvementblockenas4_ (
  Parameter batchnorm4_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm4_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm4_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm4_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter qconv4_weight (shape=(64, 64, 3, 3), dtype=<class 'numpy.float32'>)
)

In [42]:
ibe2 = ImprovementBlockEnas(64,64, params=ibe.collect_params())

In [43]:
ibe2.collect_params()

improvementblockenas4_ (
  Parameter batchnorm5_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm5_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm5_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm5_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter qconv5_weight (shape=(64, 64, 3, 3), dtype=<class 'numpy.float32'>)
)

In [55]:
ibe.collect_params('.*weight')['qconv4_weight'].data() == ibe2.collect_params('.*weight')['qconv5_weight'].data()


[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 ...


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0.

In [38]:
ibe.collect_params('.*weight')['qconv0_weight'].data().shape

(64, 64, 3, 3)

In [50]:
seq = nn.HybridSequential()
seq.add(ibe)
seq.add(ibe2)

In [51]:
seq.initialize()

In [49]:
seq.hybridize()

TypeError: hybridize() missing 1 required positional argument: 'self'

In [57]:
net = nn.Sequential()
# We need to give the shared layer a name such that we can reference
# its parameters
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

x = nd.random.uniform(shape=(2, 20))
net(x)

# Check whether the parameters are the same
print(net[1].weight.data() is net[2].weight.data())
net[1].weight.data()[0,0] = 100
# And make sure that they're actually the same object rather
# than just having the same value
print(net[1].weight.data()[0] == net[2].weight.data()[0])

True

[1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 8 @cpu(0)>


In [58]:
net[0].collect_params()

dense5_ (
  Parameter dense5_weight (shape=(8, 20), dtype=float32)
  Parameter dense5_bias (shape=(8,), dtype=float32)
)

In [62]:
net[2].collect_params()

dense4_ (
  Parameter dense4_weight (shape=(8, 8), dtype=float32)
  Parameter dense4_bias (shape=(8,), dtype=float32)
)